# Spatial Analysis and Machine Learning

In [ ]:
# import required packages 
import geopandas as gpd
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

import matplotlib.pyplot as plt

# import the required machine learning packages
from sklearn import cluster
from sklearn.preprocessing import scale

# set seaborn plotting theme to white
sns.set_theme(style="white")

## Data Exploration and Preparation
### Read in the Priority Places for Food Index (PPFI) data 

In [ ]:
PPFI = pd.read_csv('../data/week_20/ppfi_index_v2_nov2023.csv')

In [ ]:
PPFI.head()

Write a few lines of code to explore the data

In [ ]:
# explore data types

### Read in the spatial dataframe

Read in the LSOA shp file for Leeds


In [ ]:
leeds_shp =gpd.read_file('../data/week_20/Leeds.geojson')

In [ ]:
leeds_shp.head()

I have included the code below for generating the Leeds shp file, for reference for your final project.
We will talk more about finding and downloading data in the coming weeks

In [ ]:
# # Data downloaded from https://geoportal.statistics.gov.uk/maps/761ecd09b4124843b95511a242e2b1a1 
# shp =gpd.read_file('/Users/fran/Downloads/Lower_layer_Super_Output_Areas_2021_EW_BFE_V9_-3647710721716634062.geojson')
# leeds_shp =shp.loc[shp['LSOA21NM'].str.contains('Leeds'),:]	
# leeds_shp.to_file('Leeds.geojson')

### Explore, the spatial dataframe

In [ ]:
leeds_shp.explore()

### Join the PPFI data to the spatial dataframe of Leeds

In [ ]:
# join the PPFI data to the leeds_shp geodataframe using a lefthand join, on the common ID 'LSOA21CD'/'lsoa21cd'
leeds_PPFI =leeds_shp.merge(PPFI, how='left',left_on='LSOA21CD',right_on='lsoa21cd')

In [ ]:
leeds_PPFI.columns

In [ ]:
# map to check it has worked

# look at where is higher risk of Food Insecurity, don't forget smaller numbers = 'higher priority' 
leeds_PPFI.explore('pp_dec_combined', cmap='Reds_r')

### Maping the Priority Places for Food Index Deciles in Leeds

In [ ]:
# create a list of the columns we want to plot
decile_cols =['pp_dec_domain_supermarket_proximity',
       'pp_dec_domain_supermarket_accessibility',
       'pp_dec_domain_ecommerce_access', 'pp_dec_domain_socio_demographic',
       'pp_dec_domain_nonsupermarket_proximity',
       'pp_dec_domain_food_for_families', 'pp_dec_domain_fuel_poverty']

# create a for loop for plotting a map of each domain 
# this saves writing out 7 blocks of code for each visualisation

# for each item in a range from 0 to 7 (number of items in the deciles_cols list)
for i in range (0, len(decile_cols)):
    
    # produce a plot
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    
    # get the ith item in the decile column list and plot
    leeds_PPFI.plot(column=decile_cols[i],
    
  
    #### format the plot ###
                    
    # reduce linewidth between polygons
    linewidth =0.1,                
    # specify data is categorical (ordinal)                
    categorical=True,                 
    # show the legend
    legend=True, 
    # define the legend palette
    cmap='RdBu', 
    # use the define axis                
    ax=ax,
    # position the legend         
    legend_kwds={'loc': 'center left','bbox_to_anchor':(1,0.5)})
    
    # add a title based on the column plotted, formatting the title to look better
    plt.title(decile_cols[i].replace('_',' ').replace('pp dec domain ','').capitalize()+' domain in Leeds')
    
    # do not plot with the axis showing
    plt.axis('off')
   
    # save the figure as an image with name reflecting the domain plotted
    plt.savefig('week_20_outputs/'+str(decile_cols[i])+'_'+'Leeds'+'.jpg',bbox_inches='tight');

### Explore the association between PPFI domains 
To better understand how the domains that make up the 'combined' PPFI index are associated we can plot a pair plot (each domain against the other domains)

In [ ]:
# plot pairplot, adjust height so domain names fit on axis
sns.pairplot(leeds_PPFI[['domain_supermarket_proximity',
       'domain_supermarket_accessibility', 'domain_ecommerce_access',
       'domain_socio_demographic', 'domain_nonsupermarket_proximity',
       'domain_food_for_families', 'domain_fuel_poverty']], height=4)

#### Quantify the association between PPFI domains using Spearman's rank correlation

In [ ]:
# Calculate Spearman's rank correlation
ppfi_domains_corr =leeds_PPFI[['domain_supermarket_proximity',
       'domain_supermarket_accessibility', 'domain_ecommerce_access',
       'domain_socio_demographic', 'domain_nonsupermarket_proximity',
       'domain_food_for_families', 'domain_fuel_poverty']].corr(method = 'spearman')
ppfi_domains_corr

#### Visualise the Spearman's rank correlation of the PPFI domains

In [ ]:
# define plot size
fig,ax = plt.subplots(figsize=(8,8))

# define mask to apply to upper right hand corner of the plot
data_to_mask = np.triu(np.ones_like(ppfi_domains_corr))

# define axis tick labels
# get the index and the columns, replace the underscores with spaces and remove 'domain ' from the name
x_axis_labels = ppfi_domains_corr.columns.str.replace('_',' ').str.replace('domain ','')
y_axis_labels = ppfi_domains_corr.index.str.replace('_',' ').str.replace('domain ','')

# Have a look at what the tick labels look like
print('Labels:',x_axis_labels ,y_axis_labels)

# If we want to capitalize each tick label
# for each element in the list of x_axis_labels, capitalize it
x_axis_labels = [element.capitalize() for element in x_axis_labels]
y_axis_labels = [element.capitalize() for element in y_axis_labels]

# look at the new labels
print('Labels with capital letters:',x_axis_labels ,y_axis_labels)


# plot a heatmap of the correlation dataframe
sns.heatmap(ppfi_domains_corr, 
            # annotate so spearman's rank correlation values are displayed on the squares
            annot=True, 
            # define colourmap
            cmap='RdBu', 
            # define value of minimum colour on cbar
            vmin=-1, 
            # define value of maximum colour on cbar
            vmax=1,
            # add the mask
            mask=data_to_mask, 
            # use the custom tick labels
            xticklabels=x_axis_labels, 
            yticklabels=y_axis_labels,
            # add a label to the cbar
            cbar_kws={'label': "Spearman's Rank correlation"},
            # plot on the axis we defined
            ax=ax)

# Set axis labels
ax.set(xlabel="Priority Places for Food Index Domains", 
       ylabel="Priority Places for Food Index Domains",
      title ='Priority Places for Food Index Domain Correlation' );
plt.savefig('week_20_outputs/Priority_Places_for_Food_Index_Domain_Correlation.png')

#### Let's investigate what is going on the visualisation above.
<b>What happens if you exclude 'mask=data_to_mask'?</b>

In [ ]:
# define plot size
fig,ax = plt.subplots(figsize=(8,8))

# plot a heatmap of the correlation dataframe
sns.heatmap(ppfi_domains_corr, 
            # annotate so spearman's rank correlation values are displayed on the squares
            annot=True, 
            # define colourmap
            cmap='RdBu', 
            # define value of minimum colour on cbar
            vmin=-1, 
            # define value of maximum colour on cbar
            vmax=1,
            # comment out the mask
            #mask=data_to_mask, 
            # use the custom tick labels
            xticklabels=x_axis_labels, 
            yticklabels=y_axis_labels,
            # add a label to the cbar
            cbar_kws={'label': "Spearman's Rank correlation"},
            # plot on the axis we defined
            ax=ax)

# Set axis labels
ax.set(xlabel="Priority Places for Food Index Domains", 
       ylabel="Priority Places for Food Index Domains",
      title ='Priority Places for Food Index Domain Correlation' );


<b> Run the code below, to test what each part of the code does. </b>

In [ ]:
# np.ones_like: Returns an array of ones with the same shape and type as a given array.
np.ones_like(ppfi_domains_corr)

In [ ]:
#np.triu() returns the Upper triangle of an array

np.triu(np.ones_like(ppfi_domains_corr))

In the function <code>sns.heatmap(mask) </code>, <code> mask<code> 'If passed, data will not be shown in cells where mask is True" ie.e '1'

So in out <code>np.triu(np.ones_like(ppfi_domains_corr))</code> array, only values were the array is 0 will be shown in the heatmap

#### Comment out different parts of the code below to test you know what each parameter in the funciton is doing

Look at the package documentation (https://seaborn.pydata.org/archive/0.11/generated/seaborn.heatmap.html) to see if you can add further features to the visualisation 

e.g. make the cbar horizontal or increase the line wdith between heatmap squares

In [ ]:
# a copy of the code is here for you to explore what the different parameters do 

# define plot size
fig,ax = plt.subplots(figsize=(8,8))
# plot a heatmap of the correlation dataframe
sns.heatmap(ppfi_domains_corr, 
            # annotate so spearman's rank correlation values are dispalyed on the squares
            annot=True, 
            # define colourmap
            cmap='RdBu', 
            # define value of minimum colour on cbar
            vmin=-1, 
            # define value of maximum colour on cbar
            vmax=1,
            # add the mask
            mask=data_to_mask, 
            # use the custom tick labels
            xticklabels=x_axis_labels, 
            yticklabels=y_axis_labels,
            # add a label to the cbar
            cbar_kws={'label': "Spearman's Rank correlation"},
            # plot on the axis we defined
            ax=ax)

# Set axis labels
ax.set(xlabel="Priority Places for Food Index Domains", 
       ylabel="Priority Places for Food Index Domains",
      title ='Priority Places for Food Index Domain Correlation' );

## K-means clustering

### Identify the number of clusters using the elbow method

In [ ]:
# create an empty list to fill with values later
Sum_of_squared_distances = []

# get a range of numbers from 1 to 15
K = range(1,15)
#for each number in the range 1 to 15
for k in K:
    # create a k-means model with that number of clusters
    # set random state 
    km = cluster.KMeans(n_clusters=k, init="random", random_state=123)
    # fit the model using the variables of interest (in this case the 7 PPFI domain deciles)
    km = km.fit(leeds_PPFI[['pp_dec_domain_supermarket_proximity',
       'pp_dec_domain_supermarket_accessibility',
       'pp_dec_domain_ecommerce_access', 'pp_dec_domain_socio_demographic',
       'pp_dec_domain_nonsupermarket_proximity',
       'pp_dec_domain_food_for_families', 'pp_dec_domain_fuel_poverty']].values)
    # calculate the sum of the squared distances and add this to the 'Sum_of_squared_distances' list we created earlier
    Sum_of_squared_distances.append(km.inertia_)

# plot the sum of squared distances against the number of clusters
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

As discussed in the lecture we want to find the point in the graph where increasing the number of clusters does not substantively reduce the sum of squared distances (the sum of the distances of each point in the data to its' cluster centroid).
Here the 'elbow' is not too distinctive, I would argue 5 clusters would probably be best. 

### Run the K-means model with N=5 clusters

In [ ]:
# run the model with 5 clusters
km5 = cluster.KMeans(n_clusters=5,init="random", random_state=123)
km5cls = km5.fit(leeds_PPFI[['pp_dec_domain_supermarket_proximity',
       'pp_dec_domain_supermarket_accessibility',
       'pp_dec_domain_ecommerce_access', 'pp_dec_domain_socio_demographic',
       'pp_dec_domain_nonsupermarket_proximity',
       'pp_dec_domain_food_for_families', 'pp_dec_domain_fuel_poverty']].values)


In [ ]:
# let's look at the cluster labels (each LSOA has been assigned a cluster in this array)
km5cls.labels_

#### Let's create a new column with the cluster label  

In [ ]:
# let's look at the cluster labels (each LSOA has been assigned a cluster in this array)

leeds_PPFI['Cluster'] = km5cls.labels_
leeds_PPFI['Cluster'].head()

#### Create visualisation to understand the relationships captured in the clustering

In [ ]:
# The pari plot gives us an idea of all the relationships between the domains captured by the clustering
sns.pairplot(leeds_PPFI[['domain_supermarket_proximity',
       'domain_supermarket_accessibility', 'domain_ecommerce_access',
       'domain_socio_demographic', 'domain_nonsupermarket_proximity',
       'domain_food_for_families', 'domain_fuel_poverty','Cluster']],
             hue='Cluster',
             palette='Dark2',
            height=5)

### Explore the clustering results
#### Map the clusters to view any spatial relationship(s) between clusters

In [ ]:
# We can also see that there is a spatial pattern to the clusters when we map them
f, ax = plt.subplots(1, figsize=(9, 9))

leeds_PPFI.plot(column='Cluster', categorical=True, legend=True, \
         linewidth=0.1, edgecolor='white', ax=ax)

ax.set_axis_off()

plt.show()

#### Calculate summary statisitcs to compare clusters

In [ ]:
# use groupby to get the median decile value of each PPFI domain by cluster 
leeds_PPFI_clusters_median=leeds_PPFI.groupby('Cluster')[['pp_dec_domain_supermarket_proximity',
       'pp_dec_domain_supermarket_accessibility',
       'pp_dec_domain_ecommerce_access', 'pp_dec_domain_socio_demographic',
       'pp_dec_domain_nonsupermarket_proximity',
       'pp_dec_domain_food_for_families', 'pp_dec_domain_fuel_poverty']].median().reset_index()

In [ ]:
leeds_PPFI_clusters_median

In [ ]:
# let's transform the data from a 'wide' format to a 'long' format to plot
leeds_PPFI_clusters_median_to_plot =pd.melt(leeds_PPFI_clusters_median,id_vars='Cluster', 
                                            value_vars=['pp_dec_domain_supermarket_proximity',
       'pp_dec_domain_supermarket_accessibility',
       'pp_dec_domain_ecommerce_access', 'pp_dec_domain_socio_demographic',
       'pp_dec_domain_nonsupermarket_proximity',
       'pp_dec_domain_food_for_families', 'pp_dec_domain_fuel_poverty'])

In [ ]:
# check what the data now look like
leeds_PPFI_clusters_median_to_plot.head()

In [ ]:
# rename columns ot be more intuitive
leeds_PPFI_clusters_median_to_plot.columns = ['Cluster','Priority Places Domain','Median decile']

In [ ]:
# check what the data now look like
leeds_PPFI_clusters_median_to_plot.head()

#### Visualise Cluster by average (median) domain values

In [ ]:
# Plot a faceted bar chart, where each row is a different cluster

sns.catplot(leeds_PPFI_clusters_median_to_plot, 
            row='Cluster', 
            y='Priority Places Domain', 
            x='Median decile',
            kind='bar', 
            aspect=4, 
            hue='Median decile',
            palette='autumn')
plt.savefig('week_20_outputs/cluster_domains_overview.png')

### Name clusters using generated cluster summaries

Cluster '0','1' etc... are not that intuative, we can instead create a new variable and assign a Cluster description.

In [ ]:
# create empty column
leeds_PPFI['Cluster_description']=""

Wrtie your own cluster descriptions using the graph above, your clusters might look different to mine.  
e.g. <br>
<code> leeds_PPFI.loc[leeds_PPFI['Cluster']==1,'Cluster_description']='More likely to be choosing between heat or eat'</code> <br>
<code> leeds_PPFI.loc[leeds_PPFI['Cluster']==2,'Cluster_description']='Struggling to access and afford food'</code><br>
<code> leeds_PPFI.loc[leeds_PPFI['Cluster']==3,'Cluster_description']='Struggling to access supermarkets'</code><br>


In [ ]:
# edit to add your cluster descriptions
leeds_PPFI.loc[leeds_PPFI['Cluster']==0,'Cluster_description']=''
leeds_PPFI.loc[leeds_PPFI['Cluster']==1,'Cluster_description']=''
leeds_PPFI.loc[leeds_PPFI['Cluster']==2,'Cluster_description']=''
leeds_PPFI.loc[leeds_PPFI['Cluster']==3,'Cluster_description']=''
leeds_PPFI.loc[leeds_PPFI['Cluster']==4,'Cluster_description']=''


In [ ]:
# #My cluster descriptions
# leeds_PPFI.loc[leeds_PPFI['cluster']==0,'Cluster_description']='Financially constrained'
# leeds_PPFI.loc[leeds_PPFI['cluster']==1,'Cluster_description']='More likely to be choosing between heat or eat'
# leeds_PPFI.loc[leeds_PPFI['cluster']==2,'Cluster_description']='Struggling to access and afford food'
# leeds_PPFI.loc[leeds_PPFI['cluster']==3,'Cluster_description']='Struggling to access supermarkets'
# leeds_PPFI.loc[leeds_PPFI['cluster']==4,'Cluster_description']='Getting by'

### Visualise Cluster Description

In [ ]:
# use hex colours to define my own cmap, 
# cbind friendly palette colours taken from: https://davidmathlogic.com/colorblind/#%23648FFF-%23785EF0-%23DC267F-%23FE6100-%23FFB000
my_cmap=['#FFB000','#785EF0','#DC267F','#FE6100','#648FFF']

#### Map cluster descriptions

In [ ]:
# Plot your cluster descriptions on the map
map =leeds_PPFI.sort_values('Cluster_description').explore('Cluster_description', categorical =True, cmap=my_cmap)

# save to html file
map.save("week_20_outputs/leeds_PPFI_cluster_map.html")

# view map
map

### Reproduce pairplot with Cluster descriptions

In [ ]:
sns.pairplot(leeds_PPFI[['domain_supermarket_proximity',
       'domain_supermarket_accessibility', 'domain_ecommerce_access',
       'domain_socio_demographic', 'domain_nonsupermarket_proximity',
       'domain_food_for_families', 'domain_fuel_poverty','Cluster_description']].sort_values('Cluster_description'),
             hue='Cluster_description',
             palette=my_cmap,
            height=4)